<a href="https://colab.research.google.com/github/ewei2406/Diagnosis/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [309]:
import csv
import json
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import seaborn as sns

# Load and preprocess data

In [310]:
# Data sourced from Disease-Symptom database at Columbia:
# https://people.dbmi.columbia.edu/~friedma/Projects/DiseaseSymptomKB/index.html

f = open("Illness.csv")

csvreader = csv.reader(f)
next(csvreader)

df = pd.DataFrame({'Disease': [], 'Weighting': []})

for row in csvreader:
    # print(row)
    if row[0] != '':
        current_disease = row[0].split("_")[-1]
        df.loc[len(df)] = 0
        df.iloc[-1, 0] = current_disease
        df.iloc[-1, 1] = row[1]
    
    symptom = row[2].split("_")[-1]
    if symptom not in df.columns:
      df[symptom] = 0
  
    df.iloc[-1, df.columns.get_loc(symptom)] = 1

df

,Disease,Weighting,pain chest,shortness of breath,dizziness,asthenia,fall,syncope,vertigo,sweating increased,palpitation,nausea,angina pectoris,pressure chest,polyuria,polydypsia,orthopnea,rale,unresponsiveness,mental status changes,vomiting,labored breathing,feeling suicidal,suicidal,hallucinations auditory,feeling hopeless,weepiness,sleeplessness,motor retardation,irritable mood,blackout,mood depressed,hallucinations visual,worry,agitation,tremor,intoxication,verbal auditory hallucinations,energy increased,difficulty,...,breath-holding spell,scleral icterus,retch,blanch,elation,verbally abusive behavior,transsexual,behavior showing increased motor activity,coordination abnormal,choke,bowel sounds decreased,no known drug allergies,low back pain,charleyhorse,sedentary,feels hot/feverish,flare,pericardial friction rub,hoard,panic,cardiovascular event,soft tissue swelling,rhd positive,para 1,nasal flaring,sneeze,hypertonicity,Murphy's sign,flatulence,gasping for breath,feces in rectum,prodrome,hypoproteinemia,alcohol binge episode,abdomen acute,air fluid level,catching breath,large-for-dates fetus,immobile,homicidal thoughts
0,hypertensive disease,3363,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,diabetes,1421,1,1,0,1,0,0,1,1,0,1,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,depressive disorder,1337,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,coronary heart disease,1284,1,1,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,pneumonia,1029,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,ileus,56,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0
130,adhesion,57,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
131,delusion,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,1,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
132,affect labile,45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [311]:
weights = df[['Weighting']].to_numpy().astype(np.float).squeeze()
weights

array([3363., 1421., 1337., 1284., 1029.,  963.,  885.,  835.,  759.,
        685.,  630.,  597.,  544.,  524.,  504.,  445.,  408.,  405.,
        398.,  390.,  354.,  350.,  341.,  325.,  311.,  310.,  297.,
        297.,  294.,  290.,  283.,  280.,  269.,  269.,  268.,  267.,
        247.,  241.,  228.,  226.,  218.,  208.,  192.,  186.,  186.,
        179.,  172.,  171.,  169.,  168.,  166.,  165.,  165.,  165.,
        165.,  164.,  163.,  161.,  160.,  158.,  152.,  147.,  145.,
        144.,  143.,  142.,  140.,  142.,  140.,  140.,  138.,  135.,
        133.,  128.,  126.,  124.,  123.,  122.,  119.,  114.,  114.,
        114.,  113.,  111.,  108.,  105.,  104.,  103.,  101.,  101.,
         99.,   99.,   96.,   96.,   95.,   94.,   94.,   93.,   92.,
         90.,   90.,   87.,   87.,   86.,   85.,   84.,   82.,   80.,
         80.,   76.,   76.,   76.,   74.,   71.,   71.,   70.,   68.,
         68.,   68.,   68.,   68.,   67.,   67.,   66.,   61.,   61.,
         61.,   61.,

In [312]:
labels = df[["Disease"]].transpose().to_numpy()[0]
labels

array(['hypertensive disease', 'diabetes', 'depressive disorder',
       'coronary heart disease', 'pneumonia', 'failure heart congestive',
       'accident cerebrovascular', 'asthma', 'myocardial infarction',
       'hypercholesterolemia', 'infection', 'infection urinary tract',
       'anemia', 'chronic obstructive airway disease', 'dementia',
       'insufficiency renal', 'confusion', 'degenerative polyarthritis',
       'hypothyroidism', 'anxiety state', 'primary malignant neoplasm',
       'hiv infections', 'cellulitis', 'gastroesophageal reflux disease',
       'sepsis (invertebrate)', 'deep vein thrombosis', 'dehydration',
       'neoplasm', 'embolism pulmonary', 'epilepsy', 'cardiomyopathy',
       'chronic kidney failure', 'carcinoma', 'hepatitis C',
       'peripheral vascular disease', 'psychotic disorder',
       'hyperlipidemia', 'bipolar disorder', 'obesity', 'ischemia',
       'cirrhosis', 'exanthema', 'benign prostatic hypertrophy',
       'kidney failure acute', 'mitra

In [313]:
symptoms_df = df.copy()
symptoms_df = symptoms_df.drop(['Disease', 'Weighting'], axis=1)
symptoms_df

,pain chest,shortness of breath,dizziness,asthenia,fall,syncope,vertigo,sweating increased,palpitation,nausea,angina pectoris,pressure chest,polyuria,polydypsia,orthopnea,rale,unresponsiveness,mental status changes,vomiting,labored breathing,feeling suicidal,suicidal,hallucinations auditory,feeling hopeless,weepiness,sleeplessness,motor retardation,irritable mood,blackout,mood depressed,hallucinations visual,worry,agitation,tremor,intoxication,verbal auditory hallucinations,energy increased,difficulty,nightmare,unable to concentrate,...,breath-holding spell,scleral icterus,retch,blanch,elation,verbally abusive behavior,transsexual,behavior showing increased motor activity,coordination abnormal,choke,bowel sounds decreased,no known drug allergies,low back pain,charleyhorse,sedentary,feels hot/feverish,flare,pericardial friction rub,hoard,panic,cardiovascular event,soft tissue swelling,rhd positive,para 1,nasal flaring,sneeze,hypertonicity,Murphy's sign,flatulence,gasping for breath,feces in rectum,prodrome,hypoproteinemia,alcohol binge episode,abdomen acute,air fluid level,catching breath,large-for-dates fetus,immobile,homicidal thoughts
0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,1,0,0,1,1,0,1,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0
130,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
131,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
132,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Create and train model

In [314]:
inputs = keras.Input(shape=(399,), name="symptoms")
x = layers.Dense(256, activation="relu", name="dense_1")(inputs)
x = layers.Dense(256, activation="relu", name="dense_2")(x)
outputs = layers.Dense(134, activation="softmax", name="predictions")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [315]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [316]:
symptoms_df.columns

Index(['pain chest', 'shortness of breath', 'dizziness', 'asthenia', 'fall',
       'syncope', 'vertigo', 'sweating increased', 'palpitation', 'nausea',
       ...
       'feces in rectum', 'prodrome', 'hypoproteinemia',
       'alcohol binge episode', 'abdomen acute', 'air fluid level',
       'catching breath', 'large-for-dates fetus', 'immobile',
       'homicidal thoughts'],
      dtype='object', length=399)

In [317]:
pd.DataFrame(list(range(134))).transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133


In [318]:
model.fit(
    x=symptoms_df,
    y=pd.DataFrame(list(range(134))),
    epochs=10
)

Epoch 1/10
5/5 [==============================] - 0s 5ms/step - loss: 4.9396 - sparse_categorical_accuracy: 0.0000e+00
Epoch 2/10
5/5 [==============================] - 0s 5ms/step - loss: 4.6352 - sparse_categorical_accuracy: 0.3582
Epoch 3/10
5/5 [==============================] - 0s 4ms/step - loss: 4.4039 - sparse_categorical_accuracy: 0.6791
Epoch 4/10
5/5 [==============================] - 0s 5ms/step - loss: 4.1521 - sparse_categorical_accuracy: 0.8582
Epoch 5/10
5/5 [==============================] - 0s 7ms/step - loss: 3.8723 - sparse_categorical_accuracy: 0.8955
Epoch 6/10
5/5 [==============================] - 0s 4ms/step - loss: 3.5662 - sparse_categorical_accuracy: 0.9478
Epoch 7/10
5/5 [==============================] - 0s 4ms/step - loss: 3.2312 - sparse_categorical_accuracy: 0.9104
Epoch 8/10
5/5 [==============================] - 0s 4ms/step - loss: 2.8927 - sparse_categorical_accuracy: 0.9478
Epoch 9/10
5/5 [==============================] - 0s 4ms/step - loss: 2.5365

# Test sample user input

In [319]:
import re

In [320]:
symptoms_df.columns

Index(['pain chest', 'shortness of breath', 'dizziness', 'asthenia', 'fall',
       'syncope', 'vertigo', 'sweating increased', 'palpitation', 'nausea',
       ...
       'feces in rectum', 'prodrome', 'hypoproteinemia',
       'alcohol binge episode', 'abdomen acute', 'air fluid level',
       'catching breath', 'large-for-dates fetus', 'immobile',
       'homicidal thoughts'],
      dtype='object', length=399)

In [326]:
input_array = [0] * 399 

user_phrase = "I am wheezing and coughing."
user_phrase = re.sub("[^0-9a-zA-Z]+", " ", user_phrase).lower().split()
for word in user_phrase:
  for x in symptoms_df.columns:
    if word in x.strip('and').split():
      idx = symptoms_df.columns.get_loc(x)
      input_array[idx] = 1

input_array = pd.DataFrame(input_array).transpose()
input_array

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [327]:
res = model.predict(pd.DataFrame(input_array)) * np.log(weights)
# print(res, res.argmax(), res[0][res.argmax()])
labels[res.argmax()]

'pneumonia'

# Save the model

In [323]:
!pip install tensorflowjs

In [324]:
import tensorflowjs as tfjs

In [325]:
model.compile()
tfjs.converters.save_keras_model(model, 'saved_model')